In [2]:
import json
from tqdm import tqdm
from malaysian_sft import accept

In [1]:
from huggingface_hub import hf_hub_download
import fasttext

filename = hf_hub_download(
    repo_id="mesolitica/fasttext-language-detection-bahasa-en", 
    filename="fasttext.ftz"
)
lang_model = fasttext.load_model(filename)

In [4]:
files = [
    'reply-filter-social-media.jsonl',
    'reply-filter-social-media-questions.jsonl',
]

social_media = []

for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            splitted = l['question'].split()
            splitted = [s for s in splitted if len(s) > 3]

            if len(l['question'].split()) < 15:
                continue

            if '/' in l['answer']:
                continue

            if 'tuan/puan' in l['answer'].lower():
                continue

            if '_' in l['answer']:
                continue

            if 'Anda' in l['answer']:
                continue

            if 'Yang Berhormat' in l['answer']:
                continue

            if 'tuan' in l['answer'].lower():
                continue

            if 'puan' in l['answer'].lower():
                continue

            if 'https' in l['question']:
                continue

            if not accept(l['answer']):
                continue

            social_media.append([
                {'role': 'user', 'content': l['question']},
                {'role': 'assistant', 'content': l['answer']}
            ])

social_media_filtered = [s for s in social_media if lang_model.predict([s[-1]['content'].replace('\n', ' ')])[0][0][0] == '__label__bahasa']
filtered = []
for s in social_media_filtered:
    l = s[-1]['content'].lower()
    if 'hidup saya' in l:
        continue

    if 'possible response' in l:
        continue

    if 'continue the conversation' in l:
        continue

    if 'me:' in l:
        continue

    filtered.append(s)

374783it [00:34, 10988.08it/s]
435674it [01:26, 5053.32it/s]


In [6]:
filtered[0]

[{'role': 'user',
  'content': 'Bru 5hb weyy!! Habess dahh gaji knapa bulan ni lajuuu namateyyy habess ni. Nk cekau mne lg ni weyyy'},
 {'role': 'assistant',
  'content': 'Terima kasih atas perkongsian anda mengenai kegembiraan dan keseronokan menghadapi bulan ini! Memanglah, bulan ini dipenuhi dengan pelbagai aktiviti dan acara yang menyeronokkan, tidak kira sama ada ia berbentuk perayaan, hiburan atau pun keseronokan bersama-sama keluarga dan rakan-rakan.\n\nSaya dapat merasakan keseronokan dan kegembiraan yang melampau daripada perkongsian anda, dan saya berharap agar keseronokan ini dapat diteruskan sepanjang bulan ini. Pastinya, setiap orang mempunyai cara sendiri untuk menikmati dan menghargai waktu yang berharga ini.\n\nNamun, saya juga ingin menasihati agar kita tidak melupakan tanggungjawab dan kewajipan kita sehari-hari, seperti kerja, kuliah atau pun aktiviti lain yang perlukan perhatian kita. Penting untuk mencari keseimbangan antara keseronokan dan tanggungjawab, agar kita

In [7]:
instructions = []
for f in filtered:
    d = {
        'prompt_input': None,
        'input': f[0]['content'],
        'output': f[1]['content'],
    }
    instructions.append(d)

In [8]:
import pandas as pd

df = pd.DataFrame(instructions)
df.to_parquet('prepared-llama3-70b-social-media-qa.parquet')

In [9]:
from huggingface_hub import HfApi
api = HfApi()

api.upload_file(
    path_or_fileobj="prepared-llama3-70b-social-media-qa.parquet",
    path_in_repo='data/llama3_70b_social_media_qa-00000-of-00001.parquet',
    repo_id="mesolitica/Malaysian-SFT",
    repo_type="dataset",
)

prepared-llama3-70b-social-media-qa.parquet:   0%|          | 0.00/61.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/mesolitica/Malaysian-SFT/commit/66e17cb75bd09d8756362a4ec9b6aa4fd1654947', commit_message='Upload data/llama3_70b_social_media_qa-00000-of-00001.parquet with huggingface_hub', commit_description='', oid='66e17cb75bd09d8756362a4ec9b6aa4fd1654947', pr_url=None, pr_revision=None, pr_num=None)